In [2]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load in dataset
df = pd.read_csv('/content/drive/My Drive/differences.csv', sep=',')
pd.set_option('display.max_columns', None)

x = df.iloc[:, 2:len(df.columns)]  # Get x values of the data
y = df[['blue_win']] # Get target value (blue win)\\

# Split into training and testing sets

# Data Preprocessing




       MinionDiff  JgMinionDiff  LevelDiff  KillDiff  HeraldDiff  DragonDiff  \
0               3             9       -0.6        -6           1          -2   
1              -4           -56       -0.4         1           0          -2   
2              53             8        0.0         6           1           2   
3            -133           -47       -1.2        -9          -1          -2   
4              42            50        1.0         8           1           2   
...           ...           ...        ...       ...         ...         ...   
48646           7           -16       -0.6        -7           0           1   
48647          -4             4       -0.4         2          -1           1   
48648         -36            -4       -0.6        -9          -1          -2   
48649          19            21        0.6         2           0           2   
48650         -33             0       -0.4        -6          -1           1   

       TowersDiff  
0               0  